# Estimating SB ages using an MC approach 
---
McDermid et al. (2015) state that they use a Monte Carlo approach to estimate 1$\sigma$ errors on their mass-weighted age estimates. To do this they use ppxf *without* regularisation, but to estimate the mass-weighted age they do use regularisation.

Here, we are going to try using an MC approach, but without regularisation, to see whether we can accurately recover the mass-weighted age of the young component in the stellar population. If this works, then we can do away with regulariation, as it is very computationally expensive. 

To summarise, we will
1. Define the "truth" SFH and generate the corresponding spectrum.
2. In each MC iteration,
    1. add *additional* random noise to the spectrum.
    2. run ppxf. 
    3. compute the mass-weighted age of the young component. 
3. From the ensemble of mass-weighted age measurements, compute the mean and standard deviation. Do these rouhgly correspond to the input SFH?


In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [3]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
SNR = 100
sigma_star_kms = 250
z = 0.01

In [5]:
###########################################################################
# Generate the input SFH
###########################################################################
# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

# Simple Gaussian SFH
xx, yy = np.meshgrid(range(N_ages), range(N_metallicities))
x0 = 15
y0 = 1.5
sigma_x = 1
sigma_y = 0.1
sfh_young = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
               np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_young /= np.nansum(sfh_young)
sfh_mw_young = sfh_young * 1e8

x0 = 60
y0 = 0
sigma_x = 1
sigma_y = 0.5
sfh_old = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
             np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_old /= np.nansum(sfh_old)
sfh_mw_old = sfh_old * 1e10

# Add the young & old components
sfh_mw_original = sfh_mw_old + sfh_mw_young

# # Instead of a Gaussian SFH, try with a "delta function" SFH (i.e., 1 old & 1 young template)
sfh_mw_original = np.zeros((N_metallicities, N_ages))
sfh_mw_original[2, 4] = 0.5e7
sfh_mw_original[1, 4] = 1.5e7
sfh_mw_original[0, 20] = 1e8
sfh_mw_original[1, -3] = 1e10

sfh_mw_1D_original = np.nansum(sfh_mw_original, axis=0)


In [73]:
###########################################################################
# Function for computing the mass-weighted age 
###########################################################################
def calculate_mw_age(sfh_mw, age_thresh, ages):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    # Find the index of the threshold age in the template age array
    age_thresh_idx = np.nanargmin(np.abs(ages - age_thresh))
    
    # Compute the mass-weighted age 
    log_age_mw = np.nansum(sfh_mw_1D[:age_thresh_idx] * np.log10(ages[:age_thresh_idx])) / np.nansum(sfh_mw_1D[:age_thresh_idx])
    
    # Compute the corresponding index in the array (useful for plotting)
    log_age_mw_idx = (log_age_mw - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
    
    return log_age_mw, log_age_mw_idx
    

In [78]:
# Test that it works 
log_age_mw_input, log_age_mw_input_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.step(range(N_ages), sfh_mw_1D_original, color="black", where="mid")
ax.axvline(log_age_mw_idx, color="red")
ax.set_ylim([1, None])
ax.set_yscale("log")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## MONTE CARLO TESTING
---
Each iteration, use the same spectrum, but with random noise added to it. 

In [8]:
###########################################################################
# Generate the spectrum
###########################################################################
spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:37: RuntimeWarning: divide by zero encountered in log10
  m = ax.imshow(np.log10(sfh_mass_weighted), cmap="cubehelix_r",
/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:39: RuntimeWarning: divide by zero encountered in log10
  vmin=0, vmax=np.nanmax(np.log10(sfh_mass_weighted)))


In [10]:
###########################################################################
# Run ppxf WITH regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec_original, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              isochrones="Padova",
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")


----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 3.12 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3882...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 56.50 s
Iteration 1: optimal regul = 0.00; Δm = 0.0370655; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 40.38 s
Iteration 2: optimal regul = 100.00; Δm = 1.25046e+10; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 25.482
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 47.66 s
Iteration 3: optimal regul = 60.00; Δm = 1.64568e+09; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 5.073
---------

In [11]:
###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Helper function for multiprocessing
def ppxf_helper(seed):
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_original_err)
    spec = spec_original + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec[0] = -9999
    spec[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp
     
# Input arguments
niters = 100
nthreads = 20
args_list = list(np.random.randint(low=0, high=100 * niters, size=niters))

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Elapsed time in ppxf: {time() - t:.2f} s")


Running ppxf on 20 threads...



Elapsed time in ppxf: 167.51 s


In [79]:
###########################################################################
# Compute the mass-weighted age 
###########################################################################
log_age_mw_regul, log_age_mw_regul_idx = calculate_mw_age(pp_regul.weights_mass_weighted, age_thresh=1e9, ages=ages)

log_age_mw_list = []
for pp in pp_list:
    log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
    log_age_mw_list.append(log_age_mw)

log_age_mw_MC = np.nanmean(log_age_mw_list)
log_age_mw_MC_err = np.nanstd(log_age_mw_list)
# For plotting purposes, figure out the approx. index of these ages in the age array
log_age_mw_MC_idx = (log_age_mw_MC - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
log_age_mw_MC_err_idx = log_age_mw_MC_err / (np.log10(ages[1]) - np.log10(ages[0]))

print(f"MC method:    mean mass-weighted age (log yr) = {np.nanmean(log_age_mw_list):.2f} ± {np.nanstd(log_age_mw_list):.2f}")
print(f"Regul method: mean mass-weighted age (log yr) = {log_age_mw_regul:.2f}")
print(f"Input value:  mean mass-weighted age (log yr) = {log_age_mw_input:.2f})")

MC method:    mean mass-weighted age (log yr) = 7.45 ± 0.06
Regul method: mean mass-weighted age (log yr) = 7.54
Input value:  mean mass-weighted age (log yr) = 7.47)


In [124]:
plt.close("all")

In [131]:
###########################################################################
# COMPARE THE INPUT AND OUTPUT
###########################################################################
sfh_fit_mw_list = []
sfh_fit_lw_list = []
sfh_fit_mw_1D_list = []
sfh_fit_lw_1D_list = []

for pp in pp_list:
    sfh_fit_mw_list.append(pp.weights_mass_weighted)
    sfh_fit_lw_list.append(pp.weights_light_weighted)
    sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
    sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))
    
# Compute the mean SFH 
sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)

sfh_fit_mw_1D_regul = np.nansum(pp_regul.weights_mass_weighted, axis=0)
sfh_fit_lw_1D_regul = np.nansum(pp_regul.weights_light_weighted, axis=0)

# Plot the mass-weighted weights, summed over the metallicity dimension
for log_scale in [True, False]:
    # Create new figure 
    fig = plt.figure(figsize=(13, 4))
    ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
    ax.set_title("Mass-weighted template weights")
    
    # Plot the SFHs from each ppxf run, plus the "truth" SFH
    ax.fill_between(range(N_ages), sfh_mw_1D_original, step="mid", alpha=1.0, color="cornflowerblue", label="Input SFH")
    for jj in range(niters):
        ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
    ax.step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
    
    # Plot horizontal error bars indicating the mean mass-weighted age from (a) the MC simulations and (b) the regularised fit 
    y = 10**(0.9 * np.log10(ax.get_ylim()[1])) if log_scale else 0.9 * ax.get_ylim()[1]
    ax.errorbar(x=log_age_mw_regul_idx, y=y, xerr=0, yerr=0, 
                marker="D", mfc="lightgreen",mec="lightgreen",  ecolor="lightgreen",  
                label="MW age ($< 1$ Gyr) (regularised fit)")
    ax.errorbar(x=log_age_mw_MC_idx, y=y, xerr=log_age_mw_MC_err_idx, yerr=0, 
                marker="D", mfc="red", mec="red", ecolor="red",
                label="MW age ($< 1$ Gyr) (MC simulations)")
    ax.errorbar(x=log_age_mw_input_idx, y=y, xerr=0, yerr=0, 
                marker="D", mfc="cornflowerblue", mec="cornflowerblue", ecolor="cornflowerblue", 
                label="MW age ($< 1$ Gyr) (input)")
    
    # Decorations 
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylim([1, None])
    ax.set_ylabel(r"Template weight ($\rm M_\odot$)")
    ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
    ax.set_xlabel("Age (Myr)")
    ax.set_yscale("log") if log_scale else None

# Plot the light-weighted weights, summed over the metallicity dimension
for log_scale in [True, False]:
    # Create new figure 
    fig = plt.figure(figsize=(13, 4))
    ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
    ax.set_title("Light-weighted template weights")
    
    # Plot the SFHs from each ppxf run, plus the "truth" SFH
    for jj in range(niters):
        ax.step(range(N_ages), sfh_fit_lw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_lw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
    ax.step(range(N_ages), sfh_fit_lw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
    
    # Plot horizontal error bars indicating the mean mass-weighted age from (a) the MC simulations and (b) the regularised fit 
    y = 10**(0.9 * np.log10(ax.get_ylim()[1])) if log_scale else 0.9 * ax.get_ylim()[1]
    ax.errorbar(x=log_age_mw_regul_idx, y=y, xerr=0, yerr=0, 
                marker="D", mfc="lightgreen",mec="lightgreen",  ecolor="lightgreen",  
                label="MW age ($< 1$ Gyr) (regularised fit)")
    ax.errorbar(x=log_age_mw_MC_idx, y=y, xerr=log_age_mw_MC_err_idx, yerr=0, 
                marker="D", mfc="red", mec="red", ecolor="red",
                label="MW age ($< 1$ Gyr) (MC simulations)")
    ax.errorbar(x=log_age_mw_input_idx, y=y, xerr=0, yerr=0, 
                marker="D", mfc="cornflowerblue", mec="cornflowerblue", ecolor="cornflowerblue", 
                label="MW age ($< 1$ Gyr) (input)")
    
    # Decorations 
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    # ax.set_ylim([1, None])
    ax.set_ylabel(r"Template weight")
    ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
    ax.set_xlabel("Age (Myr)")
    ax.set_yscale("log") if log_scale else None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …